In [1]:
import os
import sys
import math
import json
import re
import random
import warnings
from pathlib import Path
from dataclasses import dataclass, asdict
from typing import Any, Dict, List, Tuple, Optional, Callable
from pprint import pprint

# Scientific stack
import numpy as np
import pandas as pd
import scipy
from scipy import signal, stats, optimize

# Audio
import librosa
import librosa.display
import soundfile as sf

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("talk")

# Machine learning
import sklearn
from sklearn import metrics, model_selection, preprocessing, decomposition
from sklearn.metrics import roc_auc_score, average_precision_score, confusion_matrix

# Deep learning
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchaudio
import tensorflow as tf

# Utilities
from tqdm import tqdm, trange
import joblib
import yaml
from dotenv import load_dotenv

# Jupyter helpers
from IPython.display import display, HTML, Audio, clear_output

In [2]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
PROJECT_ROOT = Path.cwd().parents[0] if Path.cwd().name == "notebooks" else Path.cwd()
DATA_DIR = PROJECT_ROOT / "data"
OUTPUT_DIR = PROJECT_ROOT / "outputs"

for p in [DATA_DIR, OUTPUT_DIR, OUTPUT_DIR / "models", OUTPUT_DIR / "logs", OUTPUT_DIR / "figures"]:
    p.mkdir(parents=True, exist_ok=True)

warnings.filterwarnings("ignore")

set_seed(42)
print("PROJECT_ROOT:", PROJECT_ROOT)
print("DATA_DIR:", DATA_DIR)
print("OUTPUT_DIR:", OUTPUT_DIR)

PROJECT_ROOT: c:\Users\Esmee Werk\Documents\Persoonlijke Projecten\AI-Onderzoeksvragen\research_questions\unsupervised_learning
DATA_DIR: c:\Users\Esmee Werk\Documents\Persoonlijke Projecten\AI-Onderzoeksvragen\research_questions\unsupervised_learning\data
OUTPUT_DIR: c:\Users\Esmee Werk\Documents\Persoonlijke Projecten\AI-Onderzoeksvragen\research_questions\unsupervised_learning\outputs


In [4]:
device_desc = {}

# PyTorch
if torch.cuda.is_available():
    torch_device = "cuda"
    gpu_name = torch.cuda.get_device_name(0)
    gpu_prop = torch.cuda.get_device_properties(0)
    device_desc["torch"] = {
        "device": torch_device,
        "name": gpu_name,
        "total_mem_GB": round(gpu_prop.total_memory / 1024**3, 2),
        "sm_count": gpu_prop.multi_processor_count,
        "capability": f"{gpu_prop.major}.{gpu_prop.minor}",
    }
    torch.backends.cudnn.benchmark = True
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    torch_device = "mps"  # Apple Silicon
    device_desc["torch"] = {"device": torch_device, "name": "Apple MPS"}
else:
    torch_device = "cpu"
    device_desc["torch"] = {"device": torch_device, "name": "CPU"}

# TensorFlow
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    for g in gpus:
        tf.config.experimental.set_memory_growth(g, True)
    device_desc["tensorflow"] = {"device": "GPU", "count": len(gpus)}
else:
    device_desc["tensorflow"] = {"device": "CPU", "count": 0}

print("Torch device:", device_desc["torch"])
print("TensorFlow device:", device_desc["tensorflow"])
TORCH_DEVICE = torch.device(torch_device)

# Snelle sanity check
if torch_device != "cpu":
    x = torch.randn(2, 2, device=TORCH_DEVICE)
    print("CUDA/MPS sanity tensor:", x.shape, x.device)
else:
    print("Running on CPU for PyTorch.")

def where_am_i():
    d = device_desc["torch"]
    print(f"Using PyTorch on {d['device']} - {d['name']}")
where_am_i()

Torch device: {'device': 'cpu', 'name': 'CPU'}
TensorFlow device: {'device': 'CPU', 'count': 0}
Running on CPU for PyTorch.
Using PyTorch on cpu - CPU
